## Unittest

In [5]:
%%writefile sample.py
from unittest import TestCase
class MyTests(TestCase):
    def test_one_plus_two(self):
        self.assertEqual(1 + 2, 3)

Overwriting sample.py


In [7]:
!python -m unittest sample.py

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


In [8]:
%%writefile sample.py
from unittest import TestCase
class MyTests(TestCase):
    def test_one_plus_two(self):
        self.assertEqual(1 + 2, 5)

Overwriting sample.py


In [9]:
!python -m unittest sample.py

F
FAIL: test_one_plus_two (sample.MyTests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Users\wontae\Workspace\Django\Django-Recipe-RestAPI\app\core\tests\sample.py", line 4, in test_one_plus_two
    self.assertEqual(1 + 2, 5)
AssertionError: 3 != 5

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (failures=1)


In [12]:
%%writefile sample.py
from unittest import TestCase

class MyTests(TestCase):
    def test_one_plus_two(self):
        self.assertEqual(1+2, 3)
    def test_other_assertions(self):
        self.assertTrue(1 == 1)
        self.assertFalse(1 == 2)
        self.assertGreater(2, 1)
        self.assertLess(1, 2)
        self.assertIn(1, [1,2])
        self.assertIsInstance(1, int)

Overwriting sample.py


In [14]:
!python -m unittest -v sample.py

test_one_plus_two (sample.MyTests) ... ok
test_other_assertions (sample.MyTests) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


- 매번 터미널에 긴 커맨드를 쓰기가 불편하다면 파일이 실행될 때 테스트를 실행할 수 있습니다. 파일 제일 하단에 다음과 같이 unittest 모듈의 main 메서드를 호출해주면 됩니다. <br>
- 이제 그냥 파일만 실행하면 테스트가 실행됩니다.

In [18]:
%%writefile sample.py
from unittest import TestCase, main

class MyTests(TestCase):
    def test_exceptions(self):
        with self.assertRaises(ZeroDivisionError):
            1 / 0
        with self.assertRaises(TypeError):
            1 + '2'

if __name__ == '__main__':
    main()

Overwriting sample.py


In [21]:
!python sample.py

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


## Test Mocking
1. Mock is used to change the default return value of a method at certain parts of the code <br>
2. This is mainly used in unit tests where we want to mock when certain parts of the code that require external connections. For example, we may want to mock that a database is connected without actually connecting <br>
3. This can be also used to counting the number of calls made to a method <br>

- mocking은 소외 mock이라고 불리는 가짜 객체를 생성하는 것부터 시작합니다.<br> 우리는 이 mock 객체가 어떻게 작동을 할지를 지정해줄 수 있으며, 이 mock 객체는 자신을 상대로 어떤 작업이 일어났는지를 기억합니다.

In [27]:
from unittest.mock import Mock
# return_value: return값을 정함
mock = Mock(return_value = 'Hello, Mock!')
mock()

'Hello, Mock!'

In [30]:
# side_effect: 예외 처리 담당
mock = Mock(side_effect = Exception('Oops!'))
mock()

Exception: Oops!

In [32]:
# Iteration 설정. 끝나면 에러 발생
mock = Mock(side_effect = [1, 2, 3])
print(mock())
print(mock())
print(mock())
print(mock())

1
2
3


StopIteration: 

In [34]:
# Lambda 함수 파라미터
mock = Mock(side_effect = lambda x: x * 10)
print(mock(1))
print(mock(2))

10
20


In [35]:
# 객체니까 선언 먼저하고 속성 추가도 가능
mock = Mock()
mock.return_value = 1
mock()

1

## Mock 객체 검증

In [38]:
# assert_called(): 해당 mock이 호출된 적이 있는지
mock = Mock()
mock.assert_called()

AssertionError: Expected 'mock' to have been called.

In [39]:
mock = Mock()
mock()
mock.assert_called()

In [44]:
# assert_called_once: 어떤 인자가 넘어왔는지?
mock = Mock()
mock('A', B = 'C')
mock.assert_called_with('A',B = 'C')

In [45]:
# assert_not_called: 해당 mock이 호출된 적이 없다
mock = Mock()
mock.assert_not_called()
mock()

<Mock name='mock()' id='2899469636080'>

In [46]:
# call_count
mock.call_count

1

## MagicMock
- __str__와 같은 magic method를 알아서 Mocking

In [48]:
from unittest.mock import MagicMock
mock = MagicMock()
mock.__str__.return_value

"<MagicMock id='2899473954176'>"

In [49]:
mock.__str__.return_value = "I'm a magic mock."
str(mock)

"I'm a magic mock."

## Patch
@Patch 데코레이터는 특정 모듈의 함수나 클래스를 MagicMock 인스턴스로 대체할 수 있습니다(=Patching). <br>
단위 테스트를 할 때 외부 서비스에 의존하지 않고 독립적으로 실행 가능한 단위 테스를 작성할수 있습니다.

In [53]:
%%writefile sample.py

from unittest import TestCase, main
from unittest.mock import patch

def hello():
    return 'Hello!'

class TestMe(TestCase):
    # @patch로 데코레이팅 된 함수/클래스만 Mocking
    """ 첫번째 인자: patching할 메소드를 package.module.Class.method 형태로 
    같은 모듈(py파일)에 있다면 __main__ 
    두번째 인자: return_value, 선택적
    """

    @patch('__main__.hello', return_value = "Mock!")
    # @patch 데코레이터는 MagicMock 객체를 테스트 함수의 인자로 추가합니다
    # 즉, mock_hello라는 변수에 객체를 저장합니다.
    def test_hello(self, mock_hello):
        self.assertEqual(hello(), "Mock!")
        # hello 메소드와 mock_hello 변수에 담긴 MagickMock 객체가 같은지?
        # 즉, MagicMock 객체가 제대로 hello 메소드를 Mocking했는지?
        self.assertIs(hello, mock_hello)
        mock_hello.assert_called_once_with()

if __name__ == "__main__":
    main()

Overwriting sample.py


In [54]:
!python sample.py

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


데코레이터는 외부 서비스에 의존하는 코드에 대한 테스트를 작성할 때 유용하게 쓰입니다. 예를 들어, API를 호출하는 코드에 대한 테스트를 작성할 때, 실제로 네트워크 연동을 하면 테스트가 느려지고, 깨지기 쉬워집니다.

지금부터 아래와 같이, requests 패키지를 사용하여 외부 API와 연동하여 사용자를 조회하거나 생성해주는 간단한 모듈에 대한 단위 테스트를 작성해보겠습니다.

In [ ]:
%%writefile sample.py
import requests

from unittest import TestCase
from unittest.mock import patch



# 실제 requests모듈로 response를 받아오는 함수들
def get_user(id):
    response = requests.get(f"https://jsonplaceholder.typicode.com/users/{id}")
    if response.status_code != 200:
        raise Exception("Failed to get a user")
    return response.json()

def create_user(user):
    response = requests.get(f"https://jsonplaceholder.typicode.com/users", data=user)
    if response.status_code != 201:
        raise Exception("Failed to create a user.")
    return response.json()

# Dummy User Manager
class UserManager():
    def __init__(self, name, email):
        self.id = len(self) + 1
        self.name = "Test User"
        self.email = "user@test.com"

    def get_user(self, id):


class TestUserManager(TestCase):
    @patch('requests.get')
    def test_get_user(self, mock_get):
        response = mock_get.return_value
        response.status_code = 200
        response.json.return_value = {
            'name': 'Test User',
            'email': 'user@test.com'
        }

